In [15]:
import pandas as pd
from pprint import pprint
#from caplot import Manhattan

In [16]:
df = pd.read_csv('data/variants.tsv.gz', sep='\t')

In [17]:
pprint(list(df.columns))

['locus.contig',
 'locus-position',
 'alleles',
 'rsid',
 'qual',
 'filters',
 'vep-Location',
 'vep-Allele',
 'vep-Consequence',
 'vep-IMPACT',
 'vep-SYMBOL',
 'vep-Gene',
 'vep-Feature_type',
 'vep-Feature',
 'vep-BIOTYPE',
 'vep-EXON',
 'vep-INTRON',
 'vep-HGVSc',
 'vep-HGVSp',
 'vep-cDNA_position',
 'vep-CDS_position',
 'vep-Protein_position',
 'vep-Amino_acids',
 'vep-Codons',
 'vep-Existing_variation',
 'vep-DISTANCE',
 'vep-STRAND',
 'vep-FLAGS',
 'vep-SYMBOL_SOURCE',
 'vep-HGNC_ID',
 'vep-CANONICAL',
 'vep-MANE_SELECT',
 'vep-MANE_PLUS_CLINICAL',
 'vep-TSL',
 'vep-APPRIS',
 'vep-CCDS',
 'vep-ENSP',
 'vep-SWISSPROT',
 'vep-TREMBL',
 'vep-UNIPARC',
 'vep-UNIPROT_ISOFORM',
 'vep-SIFT',
 'vep-PolyPhen',
 'vep-DOMAINS',
 'vep-HGVS_OFFSET',
 'vep-AF',
 'vep-AFR_AF',
 'vep-AMR_AF',
 'vep-EAS_AF',
 'vep-EUR_AF',
 'vep-SAS_AF',
 'vep-AA_AF',
 'vep-EA_AF',
 'vep-gnomAD_AF',
 'vep-gnomAD_AFR_AF',
 'vep-gnomAD_AMR_AF',
 'vep-gnomAD_ASJ_AF',
 'vep-gnomAD_EAS_AF',
 'vep-gnomAD_FIN_AF',
 'vep

In [18]:
df

,locus.contig,locus-position,alleles,rsid,qual,filters,vep-Location,vep-Allele,vep-Consequence,vep-IMPACT,...,LinReg2-p_value,LinReg3-n,LinReg3-sum_x,LinReg3-y_transpose_x,LinReg3-beta,LinReg3-standard_error,LinReg3-t_stat,LinReg3-p_value,REF,ALT
0,1,255923,"[""G"",""GTC""]",rs199745078,100.0,[],1:255923-255923,-,-,-,...,0.024298,2504,694.0,81.0,-0.011212,0.014870,-0.754020,0.450910,G,GTC
1,1,665125,"[""G"",""A""]",rs570403549,100.0,[],1:665125-665125,-,-,-,...,0.699500,2504,1.0,0.0,-0.031241,0.346190,-0.090241,0.928100,G,A
2,1,746897,"[""C"",""T""]",rs140286345,100.0,[],1:746897-746897,-,-,-,...,0.900130,2504,80.0,11.0,0.013625,0.040508,0.336360,0.736630,C,T
3,1,771884,"[""A"",""T""]",rs556866884,100.0,[],1:771884-771884,-,-,-,...,0.258810,2504,7.0,0.0,-0.131290,0.131870,-0.995640,0.319520,A,T
4,1,833953,"[""T"",""C""]",rs528055507,100.0,[],1:833953-833953,-,-,-,...,0.015592,2504,1.0,1.0,0.883430,0.344910,2.561300,0.010486,T,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81202,22,51071954,"[""A"",""T""]",rs538760255,100.0,[],22:51071954-51071954,-,-,-,...,0.562740,2504,2.0,0.0,-0.151470,0.244100,-0.620550,0.534950,A,T
81203,22,51106868,"[""C"",""A""]",rs568388391,100.0,[],22:51106868-51106868,-,-,-,...,0.699220,2504,1.0,0.0,-0.140940,0.345130,-0.408380,0.683030,C,A
81204,22,51110104,"[""G"",""C""]",rs143006734,100.0,[],22:51110104-51110104,-,-,-,...,0.166620,2504,24.0,1.0,-0.092322,0.069859,-1.321600,0.186440,G,C
81205,22,51195895,"[""T"",""C""]",rs562583093,100.0,[],22:51195895-51195895,-,-,-,...,0.559700,2504,4.0,1.0,0.045540,0.180000,0.253000,0.800290,T,C
